## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-22-08-56-20 +0000,bbc,'Very cool': The 70s Afro-rock genre enjoying ...,https://www.bbc.com/news/articles/cedx6vn5202o...
1,2025-12-22-08-55-00 +0000,bbc,Puppy farm ban promised in plans to improve an...,https://www.bbc.com/news/articles/c93wxd27dvko...
2,2025-12-22-08-49-47 +0000,nyt,Bondi Beach Shooting Suspects Also Used Pipe B...,https://www.nytimes.com/2025/12/22/world/austr...
3,2025-12-22-08-49-43 +0000,bbc,Bondi gunmen threw explosives at start of atta...,https://www.bbc.com/news/articles/ce86l95gx5po...
4,2025-12-22-08-37-00 +0000,wsj,Oil Gains on Venezuela Supply Risks,https://www.wsj.com/finance/commodities-future...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2337/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
80,trump,15
413,was,9
181,christmas,9
83,epstein,8
120,year,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
119,2025-12-21-20-20-00 +0000,wsj,"Before Charlie Kirk was fatally shot, he was r...",https://www.wsj.com/politics/policy/charlie-ki...,61
24,2025-12-22-06-38-22 +0000,nypost,Police claim alleged Bondi Beach gunman Naveed...,https://nypost.com/2025/12/22/world-news/polic...,45
124,2025-12-21-19-56-00 +0000,wsj,A top Justice Department official said Sunday ...,https://www.wsj.com/politics/policy/justice-de...,45
122,2025-12-21-20-10-45 +0000,nypost,Trump’s No. 2 at DOJ Todd Blanche emphatic Eps...,https://nypost.com/2025/12/21/us-news/trumps-n...,42
144,2025-12-21-17-30-34 +0000,nypost,US military targeting third tanker off coast o...,https://nypost.com/2025/12/21/world-news/us-mi...,41


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
119,61,2025-12-21-20-20-00 +0000,wsj,"Before Charlie Kirk was fatally shot, he was r...",https://www.wsj.com/politics/policy/charlie-ki...
24,45,2025-12-22-06-38-22 +0000,nypost,Police claim alleged Bondi Beach gunman Naveed...,https://nypost.com/2025/12/22/world-news/polic...
124,45,2025-12-21-19-56-00 +0000,wsj,A top Justice Department official said Sunday ...,https://www.wsj.com/politics/policy/justice-de...
144,41,2025-12-21-17-30-34 +0000,nypost,US military targeting third tanker off coast o...,https://nypost.com/2025/12/21/world-news/us-mi...
53,38,2025-12-22-02-46-39 +0000,wapo,‘60 Minutes’ correspondent says CBS’s Bari Wei...,https://www.washingtonpost.com/business/2025/1...
56,34,2025-12-22-02-15-24 +0000,nyt,Waymo Suspended Service in San Francisco After...,https://www.nytimes.com/2025/12/21/us/waymo-su...
86,31,2025-12-21-23-28-58 +0000,nypost,South Carolina town committee keeps Nativity s...,https://nypost.com/2025/12/21/us-news/south-ca...
106,29,2025-12-21-21-54-22 +0000,nypost,New Yorkers slam new anti-fare-beating turnsti...,https://nypost.com/2025/12/21/us-news/mta-inst...
38,27,2025-12-22-04-39-00 +0000,wsj,Ten-Year JGB Yield Hits 26-Year High After BOJ...,https://www.wsj.com/finance/investing/ten-year...
178,26,2025-12-21-11-00-00 +0000,wsj,Cleveland Fed President Beth Hammack said she ...,https://www.wsj.com/economy/central-banking/fe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
